# Glitch Examples

In [1]:
import sys
sys.path.insert(0, '..')
import securec
from securec import util
scope, target = util.init()

In [2]:
import os
if 'nt' in os.name:
    os.environ['PATH'] = r'C:\cw\cw\usr\bin;C:\cw\cw\home\portable\avrgcc\avr-gcc-10.1.0-x64-windows\bin;' + os.environ['PATH']

In [3]:
securec.util.compile_and_flash('./glitch_examples.c')

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2227 bytes
✓


In [4]:
scope.glitch.clk_src = 'clkgen'
scope.glitch.trigger_src = 'ext_single'
scope.glitch.repeat = 1
scope.glitch.output = "clock_xor"
scope.io.hs2 = "glitch"

In [5]:
import logging
logging.getLogger('ChipWhisperer Target').setLevel(logging.CRITICAL)
logging.getLogger('ChipWhisperer Scope').setLevel(logging.CRITICAL)

In [6]:
import itertools
from tqdm.notebook import tqdm
import numpy as np

def frange(start, end=None, inc=1):
    if isinstance(start, np.ndarray):
        return start
    if end is None:
        end = start
    return np.arange(start, end + inc, inc)

def glitchi(widths, offsets, ext_offsets, repeats, trials, write_params, read_params, success_criterion, print_reset=True, display_progress=True, challenge_success=100):
    iteration = list(itertools.product(
        frange(widths),
        frange(offsets),
        frange(ext_offsets),
        frange(repeats),
        frange(trials),
    ))
    if display_progress:
        iteration = tqdm(iteration)

    successes = []
    util.reset_target()
    for width, offset, ext_offset, repeat, _ in iteration:
        data = (width, offset, ext_offset, repeat)

        scope.glitch.width = width
        scope.glitch.offset = offset
        scope.glitch.ext_offset = ext_offset
        scope.glitch.repeat = repeat

        if scope.adc.state:
            if print_reset:
                print('reset ', *data)
            util.reset_target()

        scope.arm()
        target.simpleserial_write(*write_params)

        ret = scope.capture()
        val = target.simpleserial_read_witherrors(*read_params, glitch_timeout=0.1)

        if ret:
            if print_reset:
                print('reset ', *data)
            util.reset_target()
        else:
            if val['valid'] is False:
                if print_reset:
                    print('reset ', *data)
            else:
                if val['payload'] and success_criterion(val['payload']):
                    if challenge_success != None:
                        challenged = glitchi(
                            *data,
                            trials=frange(1, challenge_success),
                            write_params=write_params,
                            read_params=read_params,
                            success_criterion=lambda payload: payload == val['payload'],
                            print_reset=False,
                            display_progress=False,
                            challenge_success=None,
                        )
                        print('success', val['payload'], *data, '->', len(challenged) / challenge_success * 100, '%')
                        successes.append((*data, len(challenged) / challenge_success))
                    else:
                        successes.append(data)
                else:
                    pass
    return successes


## Find sweet spots

All devices behave a bit different. In order to find some sweet spots a easy glitch example can be used with broad search parameters.

In [7]:
glitchi(
    widths=frange(40, 48, 2),
    offsets=frange(-45, 0, 2),
    ext_offsets=frange(0, 10),
    repeats=frange(1, 1),
    trials=frange(1, 1),
    write_params=(0x01, b''),
    read_params=(0x01, 2),
    success_criterion=lambda payload: payload[0] != 0xf,
    print_reset=False,
    challenge_success=100,
)

  0%|          | 0/1320 [00:00<?, ?it/s]

success CWbytearray(b'0e af') 42 -45 0 1 -> 10.0 %
success CWbytearray(b'0c af') 42 -41 1 1 -> 1.0 %
success CWbytearray(b'02 af') 44 -43 9 1 -> 9.0 %
success CWbytearray(b'0e af') 46 -45 0 1 -> 7.000000000000001 %
success CWbytearray(b'02 af') 46 -45 9 1 -> 8.0 %


[(42, -45, 0, 1, 0.1),
 (42, -41, 1, 1, 0.01),
 (44, -43, 9, 1, 0.09),
 (46, -45, 0, 1, 0.07),
 (46, -45, 9, 1, 0.08)]

## Example 1

The first example addresses gives successive answers to the following questions:
- Is it possible to skip instructions? (hopefully yes ;-))
- It it possible to skip exactly one instruction?
- It it possible to skip exactly two instructions?
- How does typical "skip-patterns" look like?

To give an answer the following code shall be tested:

```c
uint16_t cnt = 0xaf00;
trigger_high();
asm("nop\n"
    "nop\n"
    "nop\n"
    "nop\n"
    "nop\n"
    "ori %0, 0x01 \n"
    "ori %0, 0x02 \n"
    "ori %0, 0x04 \n"
    "ori %0, 0x08 \n"
    "nop\n"
    "nop\n"
    "nop\n"
    "nop\n"
    "nop\n"
    : "+w"(cnt));
trigger_low();
simpleserial_put(0x01, 2, (uint8_t *)&cnt);
```

Obviously, if the result is different to `0xaf0f` an inserted glitch skipped some instruction(s).

### Test 1

Try to skip exactly one instruction.

In [9]:
glitchi(
    widths=frange(40, 46, 0.5),
    offsets=frange(-45, -40, 0.5),
    ext_offsets=frange(0, 15),
    repeats=frange(1, 5),
    trials=frange(1, 1),
    write_params=(0x01, b''),
    read_params=(0x01, 2),
    success_criterion=lambda payload: payload[0] in (0x0e, 0x0d, 0x0b, 0x07),
    print_reset=False,
    challenge_success=200,
)

  0%|          | 0/11440 [00:00<?, ?it/s]

success CWbytearray(b'0e af') 40.0 -42.5 0 2 -> 10.5 %
success CWbytearray(b'0d af') 40.0 -42.5 0 4 -> 2.0 %
success CWbytearray(b'0b af') 40.0 -42.5 0 5 -> 5.0 %
success CWbytearray(b'0d af') 40.0 -42.5 1 2 -> 7.000000000000001 %
success CWbytearray(b'0d af') 40.0 -42.5 2 1 -> 3.0 %
success CWbytearray(b'0b af') 40.0 -42.5 3 2 -> 4.0 %
success CWbytearray(b'0e af') 40.5 -43.5 1 2 -> 2.5 %
success CWbytearray(b'0b af') 40.5 -43.5 2 1 -> 6.5 %
success CWbytearray(b'0d af') 40.5 -43.5 2 5 -> 1.5 %
success CWbytearray(b'0e af') 41.5 -44.0 1 2 -> 2.5 %
success CWbytearray(b'07 af') 41.5 -44.0 1 3 -> 4.5 %
success CWbytearray(b'0b af') 41.5 -44.0 2 4 -> 8.0 %
success CWbytearray(b'07 af') 41.5 -44.0 4 1 -> 5.0 %
success CWbytearray(b'0b af') 41.5 -40.0 2 3 -> 4.5 %
success CWbytearray(b'0b af') 41.5 -40.0 2 4 -> 6.5 %
success CWbytearray(b'0e af') 42.0 -45.0 0 2 -> 9.5 %
success CWbytearray(b'0d af') 42.0 -45.0 1 2 -> 5.0 %
success CWbytearray(b'07 af') 42.0 -45.0 1 4 -> 6.0 %
success CWbyt

[(40.0, -42.5, 0, 2, 0.105),
 (40.0, -42.5, 0, 4, 0.02),
 (40.0, -42.5, 0, 5, 0.05),
 (40.0, -42.5, 1, 2, 0.07),
 (40.0, -42.5, 2, 1, 0.03),
 (40.0, -42.5, 3, 2, 0.04),
 (40.5, -43.5, 1, 2, 0.025),
 (40.5, -43.5, 2, 1, 0.065),
 (40.5, -43.5, 2, 5, 0.015),
 (41.5, -44.0, 1, 2, 0.025),
 (41.5, -44.0, 1, 3, 0.045),
 (41.5, -44.0, 2, 4, 0.08),
 (41.5, -44.0, 4, 1, 0.05),
 (41.5, -40.0, 2, 3, 0.045),
 (41.5, -40.0, 2, 4, 0.065),
 (42.0, -45.0, 0, 2, 0.095),
 (42.0, -45.0, 1, 2, 0.05),
 (42.0, -45.0, 1, 4, 0.06),
 (42.0, -45.0, 1, 5, 0.015),
 (42.0, -41.0, 0, 3, 0.01),
 (42.0, -41.0, 3, 2, 0.025),
 (42.0, -40.5, 1, 1, 0.02),
 (42.0, -40.5, 4, 2, 0.015),
 (42.5, -41.5, 1, 3, 0.025),
 (42.5, -41.5, 1, 5, 0.04),
 (42.5, -41.5, 2, 2, 0.065),
 (42.5, -41.5, 3, 3, 0.02),
 (42.5, -41.0, 2, 4, 0.035),
 (43.5, -42.0, 2, 1, 0.05),
 (43.5, -42.0, 2, 3, 0.045),
 (43.5, -42.0, 2, 4, 0.015),
 (43.5, -42.0, 3, 3, 0.035),
 (44.0, -43.0, 1, 4, 0.045),
 (44.0, -43.0, 2, 4, 0.055),
 (44.0, -42.5, 0, 1, 0.065),

### Test 2

Try to skip either the first 2 or the last 2 ORs.

In [13]:
glitchi(
    widths=frange(40, 46, 0.5),
    offsets=frange(-44, -42, 0.5),
    ext_offsets=frange(0, 5),
    repeats=frange(1, 10),
    trials=frange(1, 1),
    write_params=(0x01, b''),
    read_params=(0x01, 2),
    success_criterion=lambda payload: payload[0] in (0x03, 0x0c),
    print_reset=False,
    challenge_success=200,
)

  0%|          | 0/3900 [00:00<?, ?it/s]

success CWbytearray(b'03 af') 40.0 -42.5 1 3 -> 0.5 %
success CWbytearray(b'03 af') 40.5 -43.5 3 1 -> 1.5 %
success CWbytearray(b'03 af') 41.5 -44.0 1 9 -> 1.5 %
success CWbytearray(b'0c af') 45.0 -43.5 0 4 -> 0.5 %


[(40.0, -42.5, 1, 3, 0.005),
 (40.5, -43.5, 3, 1, 0.015),
 (41.5, -44.0, 1, 9, 0.015),
 (45.0, -43.5, 0, 4, 0.005)]

### Test 3

Try to skip 3 consecutive instructions.

In [ ]:
glitchi(
    widths=frange(40, 49, 0.5),
    offsets=frange(-45, -40, 0.5),
    ext_offsets=frange(0, 3),
    repeats=frange(10, 20),
    trials=frange(1, 1),
    write_params=(0x01, b''),
    read_params=(0x01, 2),
    success_criterion=lambda payload: payload[0] in (0x01, 0x08),
    print_reset=False,
    challenge_success=200,
)

In [ ]:
f'{0x05:04b}'

In [ ]:
hex(0x2+0x4+0x8)